# 1. Environment Setup

In [1]:
import gurobipy as gp
from gurobipy import GRB
import math
import numpy as np
import pandas as pd

# 2. Data Acquisition

### 2.1 Variables Definition

In [2]:
alloy = ['all_1','all_2','all_3','all_4','all_5']
supplier = ['sup_1','sup_2','sup_3','sup_4','sup_5']
product = ['prod_1','prod_2','prod_3']
month = ['Int', 'Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
I = len(alloy)
J = len(supplier)
Z = len(product)
M = len(month)
q = 0.05
mu = 1

In [3]:
x_name = []
for i in alloy:
    for j in supplier:
        for m in month:
            #value = 'units of '+i+' sourced from '+j+' in '+m
            value = 'order '+i+' | '+j+' | '+m
            x_name.append(value)
            
s_name = []
for m in month:
    for z in product:
        #value = 'units of '+z+' delayed by one month on '+m
        value = 'delay '+z+' | '+m
        s_name.append(value)
        
D_name = []
for m in month:
    for z in product:
        #value = 'units of '+z+' delivered to satisfy demand in '+m
        value = 'deliver '+z+' | '+m
        D_name.append(value)

lambda_1_name = []
for i in alloy:
    for j in supplier:
        for m in month:
            #value = 'if contract upper range exceeded for '+i+' from '+j+' on month '+m
            value = 'exceed '+i+' | '+j+' | '+m
            lambda_1_name.append(value)

lambda_2_name = []
for i in alloy:
    for j in supplier:
        for m in month:
            #value = 'if contract lower range exceeded for '+i+' from '+j+' on month '+m
            value = 'surrender '+i+' | '+j+' | '+m
            lambda_2_name.append(value)

### 2.2 Data Filepath

In [4]:
file_loc = 'Data Templates.xlsx'

### 2.3 Contract Upper Threshold

In [5]:
contract_upper_threshold = pd.read_excel(file_loc,sheet_name='Contract Upper Threshold',header=2, na_values=['NA'], usecols="B:F")
contract_upper_threshold.index=alloy

In [6]:
h = contract_upper_threshold
contract_upper_threshold

,Supp A,Supp B,Supp C,Supp D,Supp E
all_1,1500.0000,750.0000,1250.0000,833.3333,666.66670
all_2,10000.0000,1666.6670,5500.0000,3333.3330,833.33330
all_3,7500.0000,2500.0000,5833.3330,4208.3330,875.00000
all_4,416.6667,166.6667,375.0000,291.6667,83.33333
all_5,500.0000,191.6667,383.3333,300.0000,86.66667


### 2.4 Contract Lower Threshold

In [7]:
contract_lower_threshold = pd.read_excel(file_loc,sheet_name='Contract Lower Threshold',header=2, na_values=['NA'], usecols="B:F")
contract_lower_threshold.index=alloy

In [8]:
l = contract_lower_threshold
contract_lower_threshold

,Supp A,Supp B,Supp C,Supp D,Supp E
all_1,416.66670,583.3333,666.66670,750.0000,625.00000
all_2,5041.66700,833.3333,1666.66700,1000.0000,416.66670
all_3,1700.00000,1666.6670,2500.00000,2541.6670,858.33330
all_4,41.66667,100.0000,83.33333,108.3333,66.66667
all_5,83.33333,116.6667,88.33333,133.3333,68.33333


### 2.5 Contract Penalties

In [9]:
contract_penalties = pd.read_excel(file_loc,sheet_name='Contract Penalties',header=2, na_values=['NA'], usecols="B:F")
contract_penalties.index=['Penalty Fee %']

In [10]:
p = contract_penalties
contract_penalties

,Supp A,Supp B,Supp C,Supp D,Supp E
Penalty Fee %,1,0.6,0.9,0.7,0.44


### 2.6 Annual Contract Limit

In [11]:
annual_contracted_limit= pd.read_excel(file_loc,sheet_name='Annual Contracted Limit',header=2, na_values=['NA'], usecols="B:F")
annual_contracted_limit = annual_contracted_limit.iloc[:5,:]
annual_contracted_limit.index=alloy

In [12]:
k = annual_contracted_limit
annual_contracted_limit

,Supp A,Supp B,Supp C,Supp D,Supp E
all_1,10927.5,6556.5,4371.0,7867.8,9616.2
all_2,67200.0,33600.0,22400.0,44800.0,56000.0
all_3,17600.0,44000.0,35200.0,35200.0,44000.0
all_4,1560.0,1872.0,1768.0,3120.0,2080.0
all_5,1356.0,3616.0,2938.0,1130.0,2260.0


### 2.7 Unit Cost

In [13]:
unit_cost = pd.read_excel(file_loc,sheet_name='Unit Cost',header=2, na_values=['NA'], usecols="B:F")
unit_cost = unit_cost.iloc[:5,:]
unit_cost.index=alloy

In [14]:
c = unit_cost
unit_cost

,Supp A,Supp B,Supp C,Supp D,Supp E
all_1,275.000000,274.166667,274.583333,274.416667,273.750000
all_2,11.250000,10.666667,11.083333,10.833333,10.416667
all_3,2.333333,1.833333,2.166667,2.000000,1.666667
all_4,858.333333,855.000000,857.083333,855.833333,853.333333
all_5,210.833333,204.166667,208.333333,205.833333,202.500000


### 2.8 Product Revenue

In [15]:
prod_rev = pd.read_excel(file_loc,sheet_name='Prod Rev',header=2, na_values=['NA'], usecols="B:F")
prod_rev.index=['Revenue ($)']

In [16]:
r = prod_rev
prod_rev

,Prod 1,Prod 2,Prod 3
Revenue ($),3700,3600,5400


### 2.9 Product Recipe

In [17]:
prod_recipe= pd.read_excel(file_loc,sheet_name='Prod Recipe',header=2, na_values=['NA'], usecols="B:D")
prod_recipe = prod_recipe.iloc[:5,:]
prod_recipe.index=alloy

In [18]:
u = prod_recipe
prod_recipe

,Prod 1,Prod 2,Prod 3
all_1,21.855,10.9275,43.71
all_2,67.200,336.0000,268.80
all_3,0.000,308.0000,176.00
all_4,7.280,5.2000,6.24
all_5,5.650,11.3000,2.26


### 2.10 Predicted Demand

In [19]:
predicted_demand= pd.read_excel(file_loc,sheet_name='Predicted Demand',header=2, na_values=['NA'], usecols="B:D")
predicted_demand.index=month

In [20]:
d = predicted_demand
predicted_demand

,Prod 1,Prod 2,Prod 3
Int,0.0,0.0,0.0
Jan,48.4,11.0,13.2
Feb,44.0,13.2,16.5
Mar,33.0,15.4,16.5
Apr,35.2,11.0,16.5
May,39.6,16.5,22.0
Jun,44.0,27.5,27.5
Jul,39.6,24.2,35.2
Aug,44.0,22.0,30.8
Sep,35.2,19.8,24.2


# 3. Gurobi

In [21]:
model = gp.Model()

Academic license - for non-commercial use only - expires 2022-09-25
Using license file C:\Users\siraj\gurobi.lic


### 3.1 Decision variables:

$ x_{i,j,m} $ how much of alloy type $ i $ to source from supplier $ j $ in month $ m $

$ s_{m, z} $ how many units of product type $ z $ will be delayed by one month on month $ m $

$ D_{m, z} $ how many units of product type $ z $ were delivered to satisfy demand on month $ m $

$ \lambda^1_{i, j, m} $ a variable indicating how much the order exceeded the upper contract threshold for alloy $i$ from supplier $j$ on month $m$

$ \lambda^2_{i, j, m} $ a variable indicating how much the order fell short of the lower contract threshold for alloy $i$ from supplier $j$ on month $m$

In [22]:
x = model.addVars(I, J, M, vtype=GRB.CONTINUOUS, lb=0, name=x_name)
s = model.addVars(M,Z, vtype=GRB.CONTINUOUS, lb=0, name=s_name)
D = model.addVars(M,Z, vtype=GRB.CONTINUOUS, lb=0, name=D_name)
lambda1 = model.addVars(I, J, M, vtype=GRB.CONTINUOUS, lb=0, name=lambda_1_name)
lambda2 = model.addVars(I, J, M, vtype=GRB.CONTINUOUS, lb=0, name=lambda_2_name)

### 3.2 Objective function

Maximize 

$$
\sum \limits _{m=1} ^{12}\sum \limits _{z=1} ^{3} D_{m, z} r_{z} - \sum \limits _{m=1} ^{12}\sum \limits _{z=1} ^{3} s_{m, z} r_{z}q - \sum \limits _{i=1} ^{5}\sum \limits _{j=1} ^{5}\sum \limits _{m=1} ^{12} c_{i,j}x_{i,j,m} - \sum \limits _{i=1} ^{5}\sum \limits _{j=1} ^{5}\sum \limits _{m=1} ^{12} \lambda^{1}_{i,j,m}p_{j}c_{i,j} - \sum \limits _{i=1} ^{5}\sum \limits _{j=1} ^{5}\sum \limits _{m=1} ^{12} \lambda^{2}_{i,j,m}p_{j}c_{i,j} $$

In [23]:
revenue = sum(D[m,z]*r.iloc[0,z] for m in range(1, M) for z in range(Z))
delay = sum(s[m,z]*r.iloc[0,z]*q for m in range(1, M) for z in range(Z))
cost = sum(c.iloc[i,j]*x[i,j,m] for i in range(I) for j in range(J) for m in range(1, M))
high = sum(lambda1[i,j,m]*p.iloc[0,j]*c.iloc[i, j] for i in range(I) for j in range(J) for m in range(1, M))
low = sum(lambda2[i,j,m]*p.iloc[0,j]*c.iloc[i, j] for i in range(I) for j in range(J) for m in range(1, M))

In [24]:
model.setObjective(revenue-delay-cost-high-low,GRB.MAXIMIZE)

### 3.3 Constraints:

**Capacity**

$ \sum \limits _{m=1} ^{12} x_{i,j,m} \le k_{i,j} $ $ \forall i = 1,...,5$ and $j = 1,...5 $

In [25]:
for i in range(I):
    for j in range(J):
        model.addConstr(sum(x[i,j,m] for m in range(1, M))<=k.iloc[i,j])

**Month 0 Constraints**
    
$ x_{i,j,m=0} = s_{m=0,z} = D_{m=0} = \lambda^1_{i, j, m=0} = \lambda^2_{i, j, m=0} = 0$ $\forall i = 1, ..., 5$, $j = 1, ..., 5 $, and $ z = 1, .., 3$

In [26]:
model.addConstrs(x[i,j,0]==0 for i in range(I) for j in range(J))
model.addConstrs(lambda1[i,j,0]==0 for i in range(I) for j in range(J))
model.addConstrs(lambda2[i,j,0]==0 for i in range(I) for j in range(J))
model.addConstrs(s[0,z]==0 for z in range(Z))
model.addConstrs(D[0,z]==0 for z in range(Z))

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>}

**Enough Alloys are Sourced to Meet Delivery Requirements (no inventory passed on to next month)**
    
$\sum \limits _{j=1} ^{5} x_{i,j,m} = \sum \limits _{z=1} ^{3} D_{m,z} u_{i,z}  $ $ \forall m = 1,...,12 $ and $i = 1, ..., 5 $

In [27]:
for m in range(1, M):
    for i in range(I):
        model.addConstr(sum(x[i,j,m] for j in range(J))==sum(D[m,z]*u.iloc[i,z] for z in range(Z)))

**Maximum Delay Capacity**

$s_{m, z} \le \mu (d_{m, z}+s_{m-1, z}) $ $ \forall m = 1,...,12 $ and $ z = 1,...,3 $

In [28]:
for m in range(1, M):
    for z in range(Z):
        model.addConstr(s[m,z]<=mu*(d.iloc[m,z]+s[m-1,z]))

**Demand Delivered per Month**

$D_{m, z} = s_{m-1, z} + d_{m, z} - s_{m, z} $ $ \forall m = 1, ..., 12$ and $ z = 1, ..., 3 $

In [29]:
model.addConstrs(s[0,z]==0 for z in range(Z))

for m in range(1,M):
    for z in range(Z):
        model.addConstr(D[m,z] == s[m-1,z] + d.iloc[m,z] - s[m,z])

**Meet All Demand Predicted Within the Year**

$\sum \limits _{m=1}^{12} D_{m,z} = \sum \limits _{m=1}^{12} d_{m, z} $

In [30]:
for z in range(Z):
    model.addConstr(sum(D[m,z] for m in range(1, M)) == sum(d.iloc[m,z] for m in range(1, M)))

**Upper limit penalty**

$  x_{i,j,m} - h_{i,j} \le \lambda^1_{i, j, m} $ $\forall i = 1,...,5 $,  $ j = 1,...,5 $, and $ m = 1,...,12 $

In [31]:
for i in range(I):
    for j in range(J):
        for m in range(M):
            model.addConstr(x[i,j,m]-h.iloc[i,j] <= lambda1[i,j,m])

**Lower limit penalty**

$ l_{i,j} - x_{i,j,m} \le M \lambda^2_{i, j, m}  \forall i = 1,...,5 $, $ j = 1,...,5 $, and $ m = 1,...,12 $ where $ M $ is a very large number

In [32]:
for i in range(I):
    for j in range(J):
        for m in range(1, M):
            model.addConstr(l.iloc[i,j]-x[i,j,m] <= lambda2[i,j,m])

**Non-negativity constraints**

$ x_{i,j,m} \ge 0$ $ \forall i = 1,...,5 $, $ j = 1,...,5 $, and $ m = 1,...,12 $

$ s_{m, z} \ge 0$ $\forall m = 1,...,12 $ and $ z = 1,...,3 $

$ D_{m, z} \ge 0$ $\forall m = 1,...,12 $ and $ z = 1,...,3 $

$ \lambda^1_{i, j, m} \ge 0$  $\forall i = 1,...,5$, $ j=1, ..., 5$, and $ m = 1,...,12 $

$ \lambda^2_{i, j, m} \ge 0$  $\forall i = 1,...,5$, $ j=1, ..., 5$, and $ m = 1,...,12 $

In [33]:
# these constraints are already applied while setting up decision variables

# 4. Result

In [34]:
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 869 rows, 1053 columns and 2318 nonzeros
Model fingerprint: 0xf89ce20f
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [7e-01, 5e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 7e+04]
Presolve removed 115 rows and 84 columns
Presolve time: 0.01s
Presolved: 754 rows, 969 columns, 2169 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.5597292e+07   1.101256e+05   0.000000e+00      0s
    1382   -1.4360052e+07   0.000000e+00   0.000000e+00      0s

Solved in 1382 iterations and 0.05 seconds
Optimal objective -1.436005218e+07


In [35]:
def sen_report(model):
    print('Sensitivity Analysis (SA)\n ObjVal =', model.ObjVal)
    model.printAttr(['X', 'Obj', 'SAObjLow', 'SAObjUp'])
    model.printAttr(['X', 'RC', 'LB', 'SALBLow', 'SALBUp', 'UB', 'SAUBLow', 'SAUBUp'])
    model.printAttr(['Sense', 'Slack', 'Pi', 'RHS', 'SARHSLow', 'SARHSUp']) # Pi = shadow price
    # NOTE: printAttr prints only rows with at least one NON-ZERO value, e.g. model.printAttr('X') prints only non-zero variable values

In [36]:
sen_report(model)

Sensitivity Analysis (SA)
 ObjVal = -14360052.18157333

    Variable            X          Obj     SAObjLow      SAObjUp 
----------------------------------------------------------------
order all_1 | sup_1 | Int            0            0         -inf           -0 
order all_1 | sup_1 | Jan      416.667         -275       -428.3       -153.3 
order all_1 | sup_1 | Feb      416.667         -275       -428.3       -153.3 
order all_1 | sup_1 | Mar      416.667         -275       -428.3       -153.3 
order all_1 | sup_1 | Apr      416.667         -275       -428.3       -153.3 
order all_1 | sup_1 | May      416.667         -275       -428.3       -153.3 
order all_1 | sup_1 | Jun      416.667         -275       -428.3       -153.3 
order all_1 | sup_1 | Jul      416.667         -275       -428.3       -153.3 
order all_1 | sup_1 | Aug      416.667         -275       -428.3       -153.3 
order all_1 | sup_1 | Sep      416.667         -275       -428.3       -153.3 
order all_1 | sup_1 | O